In [1]:
%pylab
import numbers
import sys
sys.path.append(r'D:\opencv249\sources\modules\python\test')

import time
from datetime import datetime
from lxml import etree as et
from StringIO import StringIO
from primesense import openni2

import os
import cv2
import transformations as tf
'一些短别名:'
tf.e2m = tf.euler_matrix
tf.m2e = tf.euler_from_matrix
tf.q2m = tf.quaternion_matrix
tf.m2q = tf.quaternion_from_matrix
tf.e2q = tf.quaternion_from_euler
tf.q2e = tf.euler_from_quaternion
tf.m2r = tf.rotation_from_matrix
tf.r2m = tf.rotation_matrix
tf.a2q = tf.quaternion_about_axis
tf.qmul = tf.quaternion_multiply
tf.qinv = tf.quaternion_inverse

'angle <--> radian:'
a2r = np.pi / 180
r2a = 180 / np.pi

def q2a(qx, qy, qz, qw, vecLen = 4):
    '''
    参见 tf.quaternion_about_axis 实现;
    算法参见:
    http://www.euclideanspace.com/maths/geometry/rotations/conversions/angleToQuaternion/
    http://www.euclideanspace.com/maths/geometry/rotations/conversions/quaternionToAngle/
    
    测试代码：
    q=tf.a2q(0.123, (1,0,0))
    a=q2a(*q)
    print q
    print a
    '''
    if not ( isinstance(qx, numbers.Real) and isinstance(qy, numbers.Real) and isinstance(qz, numbers.Real) and isinstance(qw, numbers.Real) ):
        raise TypeError, 'Q:xyzw must be all numbers, not arrays / lists.'
        return
#     if not vecLen in (3, 4):
#         print 'vecLen 必须是 3 或者 4'
#         return
        
    angle = 2 * np.arccos(qw)
    'sin(angle/2):'
    sinA2 = np.sqrt(1 - qw * qw)
    x = qx / sinA2
    y = qy / sinA2
    z = qz / sinA2
    
    if vecLen == 4:
        return angle, x, y, z
    elif vecLen == 3:
        return x*angle, y*angle, z*angle
    else:
        raise ValueError, 'vecLen must be integer 3 or 4'
    pass


Using matplotlib backend: Qt4Agg
Populating the interactive namespace from numpy and matplotlib


In [44]:
'仅测试，解析CaptureSession.xml文件'
from datetime import datetime
from lxml import etree as et

capXmlPath = r'D:\motion-sensor\zsens\_build.vc11\bin\Release\CaptureSession.ir-chessboard.xml'
psr = et.XMLParser(remove_blank_text = True)
tree = et.parse(capXmlPath, psr)
rt = tree.getroot()
# print et.tostring(tree, pretty_print = True)
node = rt[0][0]
print node.attrib
for i, data in enumerate(node):
    print i, data.attrib['realtime'] #str, 非 time, datetime obj
    #print data.get('realtime') == data.attrib['realtime']
    dtStr = data.attrib['realtime']
    dtObj = datetime.strptime(dtStr, '%Y-%m-%d %H:%M:%S.%f')

    if i>3:
        break
    

In [6]:
'''
检查capg-imu【设备机身轴向】，方法：分别沿xyz三轴逆(+)顺(-)旋转, 观察欧拉角
'''
from datetime import datetime
from lxml import etree as et

'Y朝西。分别沿xyz三轴逆(+)顺(-)旋转, 观察欧拉角'
capXmlPath = r'D:\motion-sensor\zsens\_build.vc11\bin\Release\CaptureSession-checkaxes.xml'
'Y朝北。其余同上'
capXmlPath = r'D:\motion-sensor\zsens\_build.vc11\bin\Release\CaptureSession-checkaxes.ynorth.xml'

'Y朝北。其余同上。硬件ID：1191182427。居然plot结果是对的！与之前《姿态数据问题》报告结论相反！ //2016-1-3 04:00:34'
capXmlPath = r'D:\motion-sensor\zsens\_build.vc11\bin\Release\CaptureSession.20160103.test-duyu.xml'
# capXmlPath = r'D:\motion-sensor\zsens\_build.vc11\bin\Release\CaptureSession.20160103.test-duyu.p.xml'

'Y朝北。其余同上, 重新操作一遍, 结果同上 √ //2016-1-3 10:24:13'
capXmlPath = r'D:\motion-sensor\zsens\_build.vc11\bin\Release\CaptureSession.chkAx.xml'
# capXmlPath = r'D:\motion-sensor\zsens\_build.vc11\bin\Release\CaptureSession.chkAx.p.xml'

'Y朝北。其余同上，硬件ID: 1191182355。设备未校正，raw Rwxyz很奇怪, .p.xml看起来就好很多 //2016-1-3 10:23:36'
capXmlPath = r'D:\motion-sensor\zsens\_build.vc11\bin\Release\CaptureSession.chkAx.id1191182355.xml' 
# capXmlPath = r'D:\motion-sensor\zsens\_build.vc11\bin\Release\CaptureSession.chkAx.id1191182355.p.xml'

'Y朝北。其余同上，硬件ID: 1191182355。设备校正了！√ //2016-1-3 10:48:03'
capXmlPath = r'D:\motion-sensor\zsens\_build.vc11\bin\Release\CaptureSession.chkAx.id1191182355.calibed.xml'
capXmlPath = r'D:\motion-sensor\zsens\_build.vc11\bin\Release\CaptureSession.chkAx.id1191182355.calibed.p.xml'

'X朝北，先后单轴旋转+90°,-90°,归位。7.3°'
capXmlPath = r'D:\motion-sensor\zsens\_build.vc11\bin\Release\CaptureSession.xnorth+90.xml'
# capXmlPath = r'D:\motion-sensor\zsens\_build.vc11\bin\Release\CaptureSession.xnorth+90.p.xml'

'1191182427. old X朝北, old-xyz ±90°, new-xyz ±90°. 验证信电系坐标系与杜宇&李启雷坐标系, 数据漂移.操! 146.14°'
# capXmlPath = r'D:\motion-sensor\zsens\_build.vc11\bin\Release\CaptureSession.xnorth.oxyz.nxyz.+90-90.xml'

'''
1191182427. old X朝西(略偏北), 一边卡在固定边界, old-xyz +360°. 
发现：
    1. 坐标系确实如前所想 (xyz)->(y, -x, z); 
    2. 此设备好像raw-ori存的就是以第一帧为参考系的？怪！确实：但又对不很齐，同见 xnorth.+360.xml
'''
'err: 12.68°'
# capXmlPath = r'D:\motion-sensor\zsens\_build.vc11\bin\Release\CaptureSession.xwest.+360.xml'

'1191182427. old X朝北(略偏东), 一边卡在固定边界, old-xyz +360°, 16.25°'
# capXmlPath = r'D:\motion-sensor\zsens\_build.vc11\bin\Release\CaptureSession.xnorth.+360.xml'
'重新采, 运动如上, 起始状态差劲, 16.35°'
# capXmlPath = r'D:\motion-sensor\zsens\_build.vc11\bin\Release\CaptureSession.2427.xnorth.+360.xml'
'又重新采, 这次保存.p.xml时候程序崩溃(没存成); 可能是没有 reset-session的原因。操！, 34.2°'
# capXmlPath = r'D:\motion-sensor\zsens\_build.vc11\bin\Release\CaptureSession.2427.xnorth.+360.2.xml'
'静态, 1.1°'
# capXmlPath = r'D:\motion-sensor\zsens\_build.vc11\bin\Release\CaptureSession.2427.xnorth.static.xml'
'单独标定, 重采静态, 6.5°'
# capXmlPath = r'D:\motion-sensor\zsens\_build.vc11\bin\Release\CaptureSession.2427.xnorth.recalib.static.xml'


'''
1191182355, old X北偏东， 静止
发现：
    1. 起始不是单位阵, 偏的有点大, 见 xnorth.static.xml (X北略偏东)
    2. 关闭又打开, 换了个角度, 好点了, 见 xeast.static2.xml (X东略偏南)
    3. 仿照1重新采, 又好了, 原因不明, 见 xnorth.static3.xml
'''
# capXmlPath = r'D:\motion-sensor\zsens\_build.vc11\bin\Release\CaptureSession.2355.xnorth.static.xml'
# capXmlPath = r'D:\motion-sensor\zsens\_build.vc11\bin\Release\CaptureSession.2355.xeast.static2.xml'
# capXmlPath = r'D:\motion-sensor\zsens\_build.vc11\bin\Release\CaptureSession.2355.xnorth.static3.xml'

'以为上次采的不错, 标定误差34°那个; 但是imu首尾误差居然有138.7°, 而对应cv首尾误差23°'
capXmlPath = r'D:\Users\zhangxaochen\Documents\axxb-data\ir-cb-imu-precalib-20160103\CaptureSession.ir-cb-imu-precalib-20160103.xml'

print('capXmlPath:', capXmlPath)

psr = et.XMLParser(remove_blank_text = True)
tree = et.parse(capXmlPath, psr)
rt = tree.getroot()

node = rt[0][0]
print(node.attrib)
euls = []
agaxes = []
quas=[] #xyzw
for i, data in enumerate(node):
    attribs = data.attrib
    rxyzw = [float(attribs['Rx']), float(attribs['Ry']), float(attribs['Rz']), float(attribs['Rw'])]
    quas.append(rxyzw)
    
    eul = tf.q2e(rxyzw, axes='rxyz')
    euls.append(eul)
    
    agax = q2a(*rxyzw, vecLen=4)
    agaxes.append(agax)
    pass

quas = np.array(quas)
euls = np.array(euls)
agaxes = np.array(agaxes)

'计算误差: 0, -1帧差了多少度?'
# tf.q2m(quas[0]).T*tf.q2m(quas[-1]) #改为直接用四元数运算, 如下:
agax=q2a(*tf.qmul(tf.qinv(quas[0]), quas[-1]), vecLen=4)
print '首尾误差 0&-1 error:', agax[0]* r2a
print 'det(tf.q2m(quas[0])):', det(tf.q2m(quas[0])) #虽然有些矩阵画出来三轴，看起来不像正交，但是确实没错

# close()
figure()
subplot(311)
plot(quas)
legend(list('xyzw'))

subplot(312)
plot(euls)
legend(list('xyz'))
subplot(313)
plot(agaxes[:, 1:]) #后三个量ax-xyz
plot(agaxes[:, 0]) #angle
legend(list('xyz')+['ang'])

'---------------肉眼观察某些关键帧的mat形式'
# inds=[316, 506, 645] #CaptureSession.chkAx.id1191182355.calibed.xml
inds=[11,] #CaptureSession.xnorth+90.xml

# for i in inds:
#     if i > len(node):
#         break
#     data=node[i]
#     attribs = data.attrib
#     rxyzw = [float(attribs['Rx']), float(attribs['Ry']), float(attribs['Rz']), float(attribs['Rw'])]
#     print(tf.q2m(rxyzw)[:3, :3])
#     pass

for i in inds:
    q = quas[i]
    print 'i:', i, tf.q2m(q)[:3, :3]
    pass
    
'---------------关键帧mat矩阵不好看，改为3D绘制坐标轴，动画演示'
from mpl_toolkits.mplot3d import axes3d
fig = figure()
ax = fig.add_subplot(111, projection='3d')

'添加xyz参考轴'
o3d=(0,0,0)
oxyz=np.eye(3)
ax.plot_wireframe(*zip(o3d, oxyz[0]), color='r')
ax.plot_wireframe(*zip(o3d, oxyz[1]), color='g')
ax.plot_wireframe(*zip(o3d, oxyz[2]), color='b')

wfx = wfy = wfz = None
# for i, dat in enumerate(node):
for i, q in enumerate(quas):
#     break
#     print i
    oldwfx, oldwfy, oldwfz = wfx, wfy, wfz
    
#     attribs = dat.attrib
#     rxyzw = [float(attribs['Rx']), float(attribs['Ry']), float(attribs['Rz']), float(attribs['Rw'])]
#     rmat = tf.q2m(rxyzw)[:3, :3]
    rmat = tf.q2m(q)[:3, :3]
    
    '调试看mat'
#     if i==108:
#         print i, rmat
#     elif i==136:
#         print i, rmat

#     print(rmat)
    vx = rmat[:,0]
    vy = rmat[:,1]
    vz = rmat[:,2]
    wfx = ax.plot_wireframe(*zip((0,0,0), vx), color='r')
    wfy = ax.plot_wireframe(*zip((0,0,0), vy), color='g')
    wfz = ax.plot_wireframe(*zip((0,0,0), vz), color='b')
    if oldwfx != None:
        ax.collections.remove(oldwfx)
        ax.collections.remove(oldwfy)
        ax.collections.remove(oldwfz)
        pass
    
#     fig.suptitle(str(i), fontsize=20)
#     tit = ax.set_title(str(i))
    ax.title.set_text('idx: %s'%i)
    plt.pause(0.03)
    pass



('capXmlPath:', 'D:\\Users\\zhangxaochen\\Documents\\axxb-data\\ir-cb-imu-precalib-20160103\\CaptureSession.ir-cb-imu-precalib-20160103.xml')
{'frames': '1792', 'phyId': '1191182355'}
首尾误差 0&-1 error: 138.710457475
det(tf.q2m(quas[0])): 1.0
i:

d:\Anaconda2x86\lib\site-packages\matplotlib\collections.py:590: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  if self._edgecolors == str('face'):
d:\Anaconda2x86\lib\site-packages\matplotlib\backend_bases.py:2399: MatplotlibDeprecationWarning: Using default event loop until function specific to this GUI is implemented
  warnings.warn(str, mplDeprecation)


 11 [[ 0.97647789 -0.21076794 -0.04547316]
 [ 0.21495823  0.93511614  0.28169267]
 [-0.01684909 -0.28484149  0.95842654]]


"""
A very simple 'animation' of a 3D plot
3D动画，原文代码:
http://matplotlib.org/examples/mplot3d/wire3d_animation_demo
"""
#### from __future__ import print_function #zhangxaochen: 不喜欢 print_func
from mpl_toolkits.mplot3d import axes3d
import matplotlib.pyplot as plt
import numpy as np
import time


def generate(X, Y, phi):
    R = 1 - np.sqrt(X**2 + Y**2)
    return np.cos(2 * np.pi * X + phi) * R

fig = plt.figure()
ax = fig.add_subplot(111, projection='3d')

xs = np.linspace(-1, 1, 50)
ys = np.linspace(-1, 1, 50)
X, Y = np.meshgrid(xs, ys)
Z = generate(X, Y, 0.0)

wframe = None
tstart = time.time()
for phi in np.linspace(0, 360 / 2 / np.pi, 100):

    oldcol = wframe

    Z = generate(X, Y, phi)
    wframe = ax.plot_wireframe(X, Y, Z, rstride=2, cstride=2)

    # Remove old line collection before drawing
    if oldcol is not None:
        ax.collections.remove(oldcol)

    plt.pause(.001)

print('FPS: %f' % (100 / (time.time() - tstart)))

'''
3D画线示例代码：
https://pythonprogramming.net/3d-graphing-python-matplotlib/
'''
from mpl_toolkits.mplot3d import axes3d
import matplotlib.pyplot as plt
import numpy as np

fig = plt.figure()
ax = fig.add_subplot(111, projection='3d')
X, Y, Z = [1,2,3,4,5,6,7,8,9,10],[5,6,2,3,13,4,1,2,4,8],[2,3,3,3,5,7,9,11,9,10]
ax.plot_wireframe(X, Y, Z)

plt.show()


In [8]:
'''
使用手动挑选的帧进行手眼标定预处理！ 生成 axxb.m
依赖: 
1. xxx.oni.txt 起始绝对时间戳;
2. xxx.frames2calib.txt 中存有手动从 oni 挑选的帧: (有效性, 帧序号, 相对时间戳)
3. 从以上俩txt得到绝对时间戳, 从CaptureSession.xxx.xml 中得到对应时间的IMU数据帧;
4. 整合记录 IR+IMU数据【16列】 xxx.ImuCorresp.csv (较原始): irFid irAbsTs imuA*3 imuM*3 imuG*3 imuR*4(wxyz) imuAbsTs
5. 重新整合 IR(R,t)(来自cam.xml)+IMU(R,t) 文件, 12列, 所以是: 
'''

import os, sys
import numpy as np
import time
from datetime import datetime
from lxml import etree as et
from StringIO import StringIO


'记录oni起始绝对时间戳的txt'
# oniBegtPath = r'D:\Users\zhangxaochen\Documents\ir-chessboard.oni.txt'
'记录从oni中手动挑选的帧: (有效性, 帧序号, 相对时间戳)'
# f2cPath = r'D:\Users\zhangxaochen\Documents\ir-chessboard.oni.frames2calib.txt'
'与oni对应的IMU(zsens采集程序) XML数据'
# capXmlPath = r'D:\motion-sensor\zsens\_build.vc11\bin\Release\CaptureSession.ir-chessboard.xml'
'cv::cpp-example-calibration.exe标定的外参'
# extrFname = r'D:\Program Files (x86)\OpenNI\Samples\Bin\Release\CapturedFrames.ir-chessboard\cam.xml'

'重构，组织到一个proj中去 2015-12-13 13:03:56'
projPath = r'D:\Users\zhangxaochen\Documents\ir-cb-imu-precalib'
projPath = r'D:\Users\zhangxaochen\Documents\axxb-data\ir-cb-imu-precalib'
projPath = r'D:\Users\zhangxaochen\Documents\axxb-data\ir-cb-imu-precalib-20160103'
projPath = r'D:\Users\zhangxaochen\Documents\axxb-data\ir-cb-imu-precalib-20160111'


os.chdir(projPath)
basePath, projName = os.path.split(projPath)
print 'projName:', projName
oniPath = projName+'.oni'
print 'oniPath:', oniPath
oniBegtPath = oniPath+'.txt'
print 'oniBegtPath:', oniBegtPath
f2cPath = oniPath+'.frames2calib.txt'
print 'f2cPath:', f2cPath
capXmlPath = 'CaptureSession.%s.xml'%projName
print 'capXmlPath:', capXmlPath
capFramesFolder = os.path.join(projPath, 'CapturedFrames.'+projName)
extrFname = os.path.join(capFramesFolder, 'cam.xml')
print 'extrFname:', extrFname

oniBegt = np.asscalar(np.loadtxt(oniBegtPath))
print 'oniBegt: %d'%oniBegt
f2cPathRoot, ext = os.path.splitext(f2cPath)

'从imu-xml中找到的对应帧记录，软同步，整合记录 IR+IMU数据 16列：(irFid, irAbsTs, imu-a3m3g3r4(wxyz)+ts)，以便之后处理'
f2cImuCorrespPath = f2cPathRoot + '.ImuCorresp.csv'
cvAndImuExtrPath = f2cPathRoot + '.cvAndImuExtr.csv'

f2calib = np.loadtxt(f2cPath, delimiter = '\t', skiprows = 1)

'解析CaptureSession.xml文件'
psr = et.XMLParser(remove_blank_text = True)
tree = et.parse(capXmlPath, psr)
rt = tree.getroot()
# print et.tostring(tree, pretty_print = True)
node = rt[0][0]
print 'node.attrib:', node.attrib
'总共data条目数量'
dataCnt = len(node)

# '在xml中找到时间戳对应的帧，保存3+3+3+4+realtime数据到 txt/csv'
# with open(f2cImuCorrespPath, 'w') as f2cIrImu: #改用 np.savetxt
'去掉了原f2c txt 第一列 "ok", 并改用绝对毫秒时间戳'
csvHeader = 'irFid\t irAbsTs\t imuA*3\t imuM*3\t imuG*3\t imuR*4(wxyz)\t imuAbsTs'
csvMat = [] #之后转为np-2d-array

'假设 frames2calib.txt 记录的帧序号从前往后是按时间序的, imuCurrPos 记录当前找到第几帧了'
imuCurrPos = 0
for r, row in enumerate(f2calib):
    #print row
    '要保存的新csv一行'
    csvRow = []
    isValid, irFid, irRelTs = row #irRelTs->相对时间戳(s)
    if isValid:
        #print irFid, irRelTs
        irAbsTs = oniBegt + irRelTs * 1e3 #类似 '1448203357080'
        irAbsDt = datetime.fromtimestamp(irAbsTs / 1e3)
        #print '%d'%irAbsTs
        
        csvRow += [irFid, irAbsTs]
        
        for i in range(imuCurrPos, dataCnt - 1):
            imuData = node[i]
            attribs = imuData.attrib
            imuDtStr = attribs['realtime']
            imuDtObj = datetime.strptime(imuDtStr, '%Y-%m-%d %H:%M:%S.%f')
            
            'imu-dt转为ts:'
            #imuAbsTs = time.mktime(imuDtObj.timetuple()) #丢失毫秒
            imuAbsTs = (imuDtObj - datetime(1970,1,1)).total_seconds() * 1e3
            

            '第 i+1 条目的时间戳'
            imuDtObj2 = datetime.strptime(node[i+1].attrib['realtime'], '%Y-%m-%d %H:%M:%S.%f')

            if imuDtObj <= irAbsDt <= imuDtObj2:
                #print r, row, i
                #print imuDtObj, irAbsDt, imuDtObj2

                csvRow += [float(attribs['Ax']), float(attribs['Ay']), float(attribs['Az'])
                          ,float(attribs['Mx']), float(attribs['My']), float(attribs['Mz'])
                          ,float(attribs['Gx']), float(attribs['Gy']), float(attribs['Gz'])
                          ,float(attribs['Rw']), float(attribs['Rx']), float(attribs['Ry']), float(attribs['Rz'])
                          #,imuDtStr # < irDt
                          ,imuAbsTs #改用ts-float, 虽然肉眼难读懂
                          ]
                
                imuCurrPos = i
                break
                pass
            pass
        
        csvMat.append(csvRow)
        pass
    pass

csvMat = np.asarray(csvMat) #没有也行？！
# print csvMat

'保存到 xxx.ImuCorresp.csv文件, 不用 with-open'
np.savetxt(f2cImuCorrespPath, csvMat, fmt='%f', header=csvHeader)

'---------------5. 重新整合 IR(R,t)(来自cam.xml)+IMU(R,t) 文件, 12列'
psr = et.XMLParser(remove_blank_text = True, remove_comments=True)
tree = et.parse(extrFname, psr)
rt = tree.getroot()
cameraMatrixNode = rt[8]
# print cameraMatrixNode
distortionCoefficientsNode = rt[9]
cvExtrParamsNode = rt[12]
cvExtrParamsDataNode = cvExtrParamsNode[3]
# print extrParams.text
cvExtrParams = np.genfromtxt(StringIO(cvExtrParamsDataNode.text)) #shape=(N,2)
'IR--OpencvCalib 弄成6列, 6-tuples (rotation vector + translation vector) '
cvExtrParams = cvExtrParams.reshape(-1, 6)

'''
---------------策略①：rot-vec为轴角表示
cvExtrParams 本已经是轴角表示
需要：imu-xml的Rwxyz -> AxisAngle; t-位移fake为 (0,0,0)
可以！matlab中用 light field toolbox:: rodrigues 预处理. 详见 axxb.m
'''
imuExtrp = np.array([q2a(row[12], row[13], row[14], row[11], vecLen=3) for row in csvMat])
# print imuExtrp
# print np.hstack((imuExtrp, np.zeros_like(imuExtrp)))
imuExtrp = np.hstack([imuExtrp, np.zeros_like(imuExtrp)] )
# print imuExtrp
cvAndImuExtr = np.hstack([cvExtrParams, imuExtrp])
# print cvAndImuExtr
np.savetxt(cvAndImuExtrPath, cvAndImuExtr, fmt='%f')


'---------------策略②：rot-vec为四元数表示'
#do-sth.


projName: ir-cb-imu-precalib-20160111
oniPath: ir-cb-imu-precalib-20160111.oni
oniBegtPath: ir-cb-imu-precalib-20160111.oni.txt
f2cPath: ir-cb-imu-precalib-20160111.oni.frames2calib.txt
capXmlPath: CaptureSession.ir-cb-imu-precalib-20160111.xml
extrFname: D:\Users\zhangxaochen\Documents\axxb-data\ir-cb-imu-precalib-20160111\CapturedFrames.ir-cb-imu-precalib-20160111\cam.xml
oniBegt: 1452470775056
node.attrib: {'frames': '1912', 'phyId': '1191182355'}


'---------------\xe7\xad\x96\xe7\x95\xa5\xe2\x91\xa1\xef\xbc\x9arot-vec\xe4\xb8\xba\xe5\x9b\x9b\xe5\x85\x83\xe6\x95\xb0\xe8\xa1\xa8\xe7\xa4\xba'

In [8]:
plot(csvMat[:, 11:15])
legend('xyzw')

In [9]:
'''
验证qua-agax之间的转换, 在python/matlab 之间是否处理正确 //2016-1-3 03:48:41
结论: 假如只关心qua(而非rot, 即 +q与-q认为是同一四元数/矩阵变换), 应该正确.
'''

'wxyz'
csvMat[:, 11:15]

'xyzw, 发现倒数2, 3行四元数与 axxb.m 得到的全取了反, 他们物理意义是: ori相同, 但是rot不同(严格地说, 对于四元数插值来说)'
'http://gamedev.stackexchange.com/questions/75072/how-can-i-compare-two-quaternions-for-logical-equality'
np.hstack([csvMat[:, 12:15], csvMat[:, 11][:, None]])

'''不得已, 输出矩阵, 对比, √! 说明问题不在从 杜宇qua->agax(python, cv.Rodrigues)->qua(matlab)
是否确实因为姿态杜宇算错？ 使用raw-xml验证 //验证结果见上面cell(3D动画演示)
'''
for row in cvAndImuExtr:
    imuAgax=row[6:9]
#     print tf.q2m(tf.a2q(imuAgax))
    print cv2.Rodrigues(imuAgax)[0]

[[ 0.46057601 -0.58537632 -0.66723632]
 [ 0.79512913 -0.06201213  0.60326127]
 [-0.3945116  -0.80838671  0.43688846]]
[[ 0.14012556 -0.53023898 -0.83618864]
 [ 0.9515825  -0.16124252  0.26170901]
 [-0.27359748 -0.8323746   0.48197194]]
[[ 0.22673348 -0.83837925 -0.49569362]
 [ 0.53109453 -0.32019646  0.78448252]
 [-0.81641321 -0.44112863  0.37265937]]
[[ 0.08061846 -0.82401125 -0.56080845]
 [ 0.56275845 -0.4267556   0.7079425 ]
 [-0.82268073 -0.37267293  0.42931492]]
[[-0.46214808 -0.45822053 -0.75924508]
 [ 0.30487757 -0.8860698   0.34918473]
 [-0.83274775 -0.07010175  0.54919662]]
[[-0.58588988 -0.42930372 -0.68733643]
 [ 0.4655956  -0.8725193   0.14809055]
 [-0.66329013 -0.23325606  0.71108215]]
[[-0.03960752 -0.29272578 -0.95537577]
 [ 0.60723117 -0.76636894  0.20964005]
 [-0.79353737 -0.57183063  0.20810619]]
[[-0.05570932  0.64884435 -0.7588791 ]
 [ 0.71563505 -0.50406551 -0.48351259]
 [-0.69624919 -0.57001664 -0.43625462]]
[[-0.02160862  0.43853771 -0.89845297]
 [ 0.10246002 -0.

In [123]:
'''
已并入上文，没用了？ 记不清了 //2015-12-7 01:25:54

'''
from StringIO import StringIO

extrFname = r'D:\Program Files (x86)\OpenNI\Samples\Bin\Release\CapturedFrames\cam.xml'
psr = et.XMLParser(remove_blank_text = True, remove_comments=True)
tree = et.parse(extrFname, psr)
rt = tree.getroot()
cameraMatrixNode = rt[8]
# print cameraMatrixNode
distortionCoefficientsNode = rt[9]
extrParamsNode = rt[12]
extrParamsDataNode = extrParamsNode[3]
# print extrParams.text
extrParams = np.genfromtxt(StringIO(extrParamsDataNode.text)) #shape=(N,2)
'弄成6列, 6-tuples (rotation vector + translation vector) '
extrParams = extrParams.reshape(-1, 6)

'''
注意:
1. pcl中接受/输出的应该是相机姿态矩阵(cam->world), 但是记不准了;
2. opencv-calib得到的应该是world->cam, 可从公式推导看出
3. imu-Rwxyz得到的是什么?
'''



In [37]:
'''
2016-1-10 03:19:39
---------------1. 测试 不同stream.read_frame() 是否会互相干扰. 即, primary-stream 问题.
答: 不会干扰, 各读各的!
'''
from primesense import openni2
import numpy as np

openni2.initialize()     # can also accept the path of the OpenNI redistribution
oniFname = r'D:\Users\zhangxaochen\Documents\ir-chessboard.oni'
oniFname = r'D:\Users\zhangxaochen\Documents\axxb-data\ir-cb-imu-precalib-20160103\ir-cb-imu-precalib-20160103.oni'
dev = openni2.Device.open_file(oniFname)

depth_stream = dev.create_depth_stream()
depth_stream.start()
ir_stream = dev.create_ir_stream()
ir_stream.start()

depFcnt = depth_stream.get_number_of_frames()
for i in range(depFcnt):
    depFrame = depth_stream.read_frame()
    depfid = depFrame.frameIndex
    
    irFrame = ir_stream.read_frame()
    irfid = irFrame.frameIndex
    
    print depfid, irfid
    
    if i > 200:
        break
    pass

depth_stream.stop()
ir_stream.stop()
openni2.unload()

'''
---------------2. 测试 sleep 是否会导致 read_frame() 跳帧。 //2016-1-10 03:40:45
会的！也说明 i、 fid 未必一致
'''
import time
openni2.initialize()     # can also accept the path of the OpenNI redistribution
oniFname = r'D:\Users\zhangxaochen\Documents\ir-chessboard.oni'
dev = openni2.Device.open_file(oniFname)

depth_stream = dev.create_depth_stream()
depth_stream.start()

depFcnt = depth_stream.get_number_of_frames()
for i in range(depFcnt):
    depFrame = depth_stream.read_frame()
    depfid = depFrame.frameIndex
    '测试阻塞是否会导致 read_frame 跳帧？ 会！'
    time.sleep(1)
    print depfid
    pass

depth_stream.stop()
openni2.unload()


1 1
2 2
3 3
4 4
5 5
6 6
7 7
8 8
9 9
10 10
11 11
12 12
13 13
14 14
15 15
16 16
17 17
18 18
19 19
20 20
21 21
22 22
23 23
24 24
25 25
26 26
27 27
28 28
29 29
30 30
31 31
32 32
33 33
34 34
35 35
36 36
37 37
38 38
39 39
40 40
41 41
42 42
43 43
44 44
45 45
46 46
47 47
48 48
49 49
50 50
51 51
52 52
53 53
54 54
55 55
56 56
57 57
58 58
59 59
60 60
61 61
62 62
63 63
64 64
65 65
66 66
67 67
68 68
69 69
70 70
71 71
72 72
73 73
74 74
75 75
76 76
77 77
78 78
79 79
80 80
81 81
82 82
83 83
84 84
85 85
86 86
87 87
88 88
89 89
90 90
91 91
92 92
93 93
94 94
95 95
96 96
97 97
98 98
99 99
100 100
101 101
102 102
103 103
104 104
105 105
106 106
107 107
108 108
109 109
110 110
111 111
112 112
113 113
114 114
115 115
116 116
117 117
118 118
119 119
120 120
121 121
122 122
123 123
124 124
125 125
126 126
127 127
128 128
129 129
130 130
131 131
132 132
133 133
134 134
135 135
136 136
137 137
138 138
139 139
140 140
141 141
142 142
143 143
144 144
145 145
146 146
147 147
148 148
149 149
150 150
151 151
152 152


In [2]:
'''
---------------仅仅 read_frame, 不读实际buffer, 之后会不会保持内存, 为的是尽力避免跳帧 //2016-1-10 23:54:08
'''

from primesense import openni2

openni2.initialize()     # can also accept the path of the OpenNI redistribution
oniFname = r'D:\Users\zhangxaochen\Documents\ir-chessboard.oni'
oniFname = r'D:\Users\zhangxaochen\Documents\axxb-data\ir-cb-imu-precalib-20160103\ir-cb-imu-precalib-20160103.oni' #此文件 dep 全黑
dev = openni2.Device.open_file(oniFname)

print 'dev.get_device_info:', dev.get_device_info()
print 'dev.get_sensor_info:', dev.get_sensor_info(1)

depth_stream = dev.create_depth_stream()
depth_stream.start()
ir_stream = dev.create_ir_stream()
ir_stream.start()

dfs = []
irfs = []

depFcnt = depth_stream.get_number_of_frames()
print 'depFcnt:', depFcnt

'导致 dfs, irfs 必然长度为 depFcnt, 但可能存在尾部循环:'
for i in range(depFcnt):
    depframe = depth_stream.read_frame()
    depfid = depframe.frameIndex
    depRelTs = depframe.timestamp #微秒，非毫秒
    dfs.append(depframe)
    #print allclose(np.asarray(depframe.get_buffer_as_uint16()), 0) #对于 ir-cb-imu-precalib-20160103.oni, 得到 True
    
    irframe = ir_stream.read_frame()
    irfid = irframe.frameIndex
    irRelTs = irframe.timestamp
    irfs.append(irframe)
    
    pass

'这里不能释放, 否则下面的cell就不能用了'
# depth_stream.stop()
# openni2.unload()

dev.get_device_info: OniDeviceInfo(uri = 'D:\\Users\\zhangxaochen\\Documents\\axxb-data\\ir-cb-imu-precalib-20160103\\ir-cb-imu-precalib-20160103.oni', vendor = 'PrimeSense, Ltd.', name = 'oni File', usbVendorId = 0, usbProductId = 0)
dev.get_sensor_info: <primesense.openni2.SensorInfo object at 0x05FF9170>
depFcnt: 1157


'\xe8\xbf\x99\xe9\x87\x8c\xe4\xb8\x8d\xe8\x83\xbd\xe9\x87\x8a\xe6\x94\xbe, \xe5\x90\xa6\xe5\x88\x99\xe4\xb8\x8b\xe9\x9d\xa2\xe7\x9a\x84cell\xe5\xb0\xb1\xe4\xb8\x8d\xe8\x83\xbd\xe7\x94\xa8\xe4\xba\x86'

In [17]:
depth_stream.stop()
openni2.unload()


In [5]:
'''
处理上个cell得到的 dep/ir-frame, 读实际buffer //2016-1-10 23:54:57
用 dep-frame
播放视频演示
'''

im = None
for i, df in enumerate(dfs):
    if i > 100:
        break
    print 'i:', i
    
    depdata = df.get_buffer_as_uint16() #c_ushort_Array_307200
    depdata = np.asarray(depdata) #1D, dtype=uint16
    depdata = depdata.reshape(df.height, df.width)
    
    #imshow(depdata) #叠加绘制导致越来越慢, 弃用!
    if im is None:
        im = imshow(depdata)
    else:
        im.set_data(depdata)
    
    pause(.03)
    draw()

i: 0
i: 1
i: 2
i: 3
i: 4
i: 5
i: 6
i: 7
i: 8
i: 9
i: 10
i: 11
i: 12
i: 13
i: 14
i: 15
i: 16
i: 17
i: 18
i: 19
i: 20
i: 21
i: 22
i: 23
i: 24
i: 25
i: 26
i: 27
i: 28
i: 29
i: 30
i: 31
i: 32
i: 33
i: 34
i: 35
i: 36
i: 37
i: 38
i: 39
i: 40
i: 41
i: 42
i: 43
i: 44
i: 45
i: 46
i: 47
i: 48
i: 49
i: 50
i: 51
i: 52
i: 53
i: 54
i: 55
i: 56
i: 57
i: 58
i: 59
i: 60
i: 61
i: 62
i: 63
i: 64
i: 65
i: 66
i: 67
i: 68
i: 69
i: 70
i: 71
i: 72
i: 73
i: 74
i: 75
i: 76
i: 77
i: 78
i: 79
i: 80
i: 81
i: 82
i: 83
i: 84
i: 85
i: 86
i: 87
i: 88
i: 89
i: 90
i: 91
i: 92
i: 93
i: 94
i: 95
i: 96
i: 97
i: 98
i: 99
i: 100


d:\Anaconda2x86\lib\site-packages\matplotlib\backend_bases.py:2399: MatplotlibDeprecationWarning: Using default event loop until function specific to this GUI is implemented
  warnings.warn(str, mplDeprecation)


In [3]:
'''
处理上个cell得到的 dep/ir-frame, 读实际buffer //2016-1-10 23:54:57
用 ir-frame
用cv2在这里外定标
N图像 calibrateCamera, 一起标定, 很慢, 后面采用 solvePnP
'''

square_size = 1.
pattern_size = (5,8)
pattern_points = np.zeros( (np.prod(pattern_size), 3), np.float32 )
pattern_points[:,:2] = np.indices(pattern_size).T.reshape(-1, 2)
pattern_points *= square_size

obj_points = []
img_points = []


im = None
for i, irf in enumerate(irfs):
    if i > 20:
        break
    print 'i:', i,
    
    irdata = irf.get_buffer_as_uint16() #ir也是16, c_ushort_Array_307200; 8不行
    irdata = np.asarray(irdata) #1D, dtype=uint16
    irdata = irdata.reshape(irf.height, irf.width)
    '转为uint8灰度图:'
    irdata = irdata*255./np.max(irdata)
    irdata = irdata.astype(uint8)

    found, corners = cv2.findChessboardCorners(irdata, pattern_size)
    viz = cv2.cvtColor(irdata, cv2.COLOR_GRAY2BGR)
    cv2.drawChessboardCorners(viz, pattern_size, corners, found)
    
    if not found:
        print 'chessboard not found'
        continue
    img_points.append(corners.reshape(-1, 2))
    obj_points.append(pattern_points)

#     #imshow(depdata) #叠加绘制导致越来越慢, 弃用!
#     if im is None:
#         im = imshow(viz)#, cmap='gray')
#     else:
#         im.set_data(viz)
    
#     #pause(.03)
#     draw()
    cv2.imshow('cv2-calib', viz)
    cv2.waitKey(0)
    pass

cv2.destroyAllWindows()
print ''

'N图像一起标定, N太大会卡死(<30):'
rms, camera_matrix, dist_coefs, rvecs, tvecs = \
cv2.calibrateCamera(obj_points, img_points, (irf.width, irf.height))
print 'rms:', rms, camera_matrix

i: 0 i: 1 i: 2 i: 3 i: 4 i: 5 i: 6 i: 7 i: 8 i: 9 i: 10 i: 11 i: 12 i: 13 i: 14 i: 15 i: 16 i: 17 i: 18 i: 19 i: 20 
rms: 0.181759700341 [[ 917.76861012    0.          313.7922413 ]
 [   0.          900.63245153  236.59273852]
 [   0.            0.            1.        ]]


In [4]:
'''
接上个cell, 但是改用 solvePnP //2016-1-11 03:03:03
用 ir-frame
用cv2在这里外定标
采用 solvePnP, 而不用 calibrateCamera。
输出每一帧cv-extrinsic结果，绘制dQuas(相对第0帧)
'''

from lxml import etree as et

square_size = 1.
pattern_size = (5,8)
pattern_points = np.zeros( (np.prod(pattern_size), 3), np.float32 )
pattern_points[:,:2] = np.indices(pattern_size).T.reshape(-1, 2)
pattern_points *= square_size

# obj_points = []
# img_points = []

'+++++++++++++++预先从某 cam.xml 读入内参+畸变'
extrFname = r'D:\Users\zhangxaochen\Documents\axxb-data\ir-cb-imu-precalib-20160103\CapturedFrames.ir-cb-imu-precalib-20160103\cam.xml'
psr = et.XMLParser(remove_blank_text = True, remove_comments=True)
tree = et.parse(extrFname, psr)
rt = tree.getroot()
cameraMatrixNode = rt[8]
# print cameraMatrixNode
distortionCoefficientsNode = rt[9]
camMatDataNode = cameraMatrixNode[3]
distDataNode = distortionCoefficientsNode[3]
intrMat = np.genfromtxt(StringIO(camMatDataNode.text.replace('\n', ''))).reshape(3,3)
distoMat = np.genfromtxt(StringIO(distDataNode.text.replace('\n', '')))

'+++++++++++++++求解相机变换矩阵solvePnP'
rmat0 = None
dQuas = [] #delta-qua = R0*Ri.T

im = None
for i, irf in enumerate(irfs):
    '测试调试用 i:'
#     if i > 130:
#         break
    print 'i:', i,
    
    irdata = irf.get_buffer_as_uint16() #ir也是16, c_ushort_Array_307200; 8不行
    irdata = np.asarray(irdata) #1D, dtype=uint16
    irdata = irdata.reshape(irf.height, irf.width)
    '转为uint8灰度图:'
    irdata = irdata*255./np.max(irdata)
    irdata = irdata.astype(uint8)

    found, corners = cv2.findChessboardCorners(irdata, pattern_size)
    viz = cv2.cvtColor(irdata, cv2.COLOR_GRAY2BGR)
    cv2.drawChessboardCorners(viz, pattern_size, corners, found)
    
    if not found:
        print 'chessboard not found'
        '找不到角点时候，用前面好的帧填充：'
        if Qi is not None:
            dQuas.append(Qi)
        continue
        
    corners = corners.reshape(-1, 2)
    #     img_points.append(corners)
    #     obj_points.append(pattern_points)
    ok, rvec, tvec = cv2.solvePnP(pattern_points, corners, intrMat, distoMat)
    #print 'ok:', ok, rvec, tvec #3*1， 3*1
    
    if rmat0 is None:
        rmat0 = cv2.Rodrigues(rvec)[0]
    
    rmati = cv2.Rodrigues(rvec)[0]
    'dR = R0*Ri.T'
    dRmat = rmat0.dot(rmati.T)
    '3x3 -> 4x4'
    t = np.eye(4)
    t[:3,:3] = dRmat
    Qi = tf.m2q(t)
    dQuas.append(Qi)

#     #imshow(depdata) #叠加绘制导致越来越慢, 弃用!
#     if im is None:
#         im = imshow(viz)#, cmap='gray')
#     else:
#         im.set_data(viz)
    
#     #pause(.03)
#     draw()
    cv2.imshow('cv2-calib', viz)
    cv2.waitKey(01)
    pass

cv2.destroyAllWindows()

dQuas = np.asarray(dQuas)

'N个图像一起标定, N太大会卡死(<30):'
# rms, camera_matrix, dist_coefs, rvecs, tvecs = \
# cv2.calibrateCamera(obj_points, img_points, (irf.width, irf.height))
# print 'rms:', rms, camera_matrix

'标定数据的运动, 刻意首尾姿态尽量对齐, 查看误差'
agax=q2a(*tf.qmul(tf.qinv(dQuas[0]), dQuas[-1]), vecLen=4)
print '首尾误差 0&-1 error:', agax[0]* r2a


i: 0 i: 1 i: 2 i: 3 i: 4 i: 5 i: 6 i: 7 i: 8 i: 9 i: 10 i: 11 i: 12 i: 13 i: 14 i: 15 i: 16 i: 17 i: 18 i: 19 i: 20 i: 21 i: 22 i: 23 i: 24 i: 25 i: 26 i: 27 i: 28 i: 29 i: 30 i: 31 i: 32 i: 33 i: 34 i: 35 i: 36 i: 37 i: 38 i: 39 i: 40 i: 41 i: 42 i: 43 i: 44 i: 45 i: 46 i: 47 i: 48 i: 49 i: 50 i: 51 i: 52 i: 53 i: 54 i: 55 i: 56 i: 57 i: 58 i: 59 i: 60 i: 61 i: 62 i: 63 i: 64 i: 65 i: 66 i: 67 i: 68 i: 69 i: 70 i: 71 i: 72 i: 73 i: 74 i: 75 i: 76 i: 77 i: 78 i: 79 i: 80 i: 81 i: 82 i: 83 i: 84 i: 85 i: 86 i: 87 i: 88 i: 89 i: 90 i: 91 i: 92 i: 93 i: 94 i: 95 i: 96 i: 97 i: 98 i: 99 i: 100 i: 101 i: 102 i: 103 i: 104 i: 105 i: 106 i: 107 i: 108 i: 109 i: 110 i: 111 i: 112 i: 113 i: 114 i: 115 i: 116 i: 117 i: 118 i: 119 i: 120 i: 121 i: 122 i: 123 i: 124 i: 125 i: 126 i: 127 i: 128 i: 129 i: 130 i: 131 i: 132 i: 133 i: 134 i: 135 i: 136 i: 137 i: 138 i: 139 i: 140 i: 141 i: 142 i: 143 i: 144 i: 145 i: 146 i: 147 i: 148 i: 149 i: 150 i: 151 i: 152 i: 153 i: 154 i: 155 i: 156 i: 157 i: 1

'N\xe4\xb8\xaa\xe5\x9b\xbe\xe5\x83\x8f\xe4\xb8\x80\xe8\xb5\xb7\xe6\xa0\x87\xe5\xae\x9a, N\xe5\xa4\xaa\xe5\xa4\xa7\xe4\xbc\x9a\xe5\x8d\xa1\xe6\xad\xbb(<30):'

In [5]:
plot(dQuas)
legend(list('xyzw'))

In [32]:
'''
实验cv2.xxx 各种接口 //2016-1-11 01:33:57
D:\opencv249\sources\samples\python2\calibrate.py
'''
irdata = irf.get_buffer_as_uint16() #ir也是16, c_ushort_Array_307200; 8不行
irdata = np.asarray(irdata) #1D, dtype=uint16
irdata = irdata.reshape(irf.height, irf.width)
irdata = irdata*255./np.max(irdata)
irdata = irdata.astype(uint8)

# imshow(irdata) #红蓝热度图

pattern_size = (5,8)
found, corners = cv2.findChessboardCorners(irdata, pattern_size)
viz = cv2.cvtColor(irdata, cv2.COLOR_GRAY2BGR)
cv2.drawChessboardCorners(viz, pattern_size, corners, found)
cv2.imshow('test', viz)
cv2.waitKey(0)
cv2.destroyAllWindows()


In [11]:
'''
暂不管上面IR-IMU-calib过程，直接用 IMU 数据做初值
读取oni, depth(not IR)为主轴, 提取imu-xml对应时间的数据帧
用到 https://pypi.python.org/pypi/primesense
'''
import numpy as np
from datetime import datetime
from lxml import etree as et
from primesense import openni2

'与oni对应的IMU(zsens采集程序) XML数据'
capXmlPath = r'D:\motion-sensor\zsens\_build.vc11\bin\Release\CaptureSession.ir-chessboard.xml'
capXmlPath = r'D:\Users\zhangxaochen\Documents\axxb-data\ir-cb-imu-precalib-20160103\CaptureSession.ir-cb-imu-precalib-20160103.xml'
'解析CaptureSession.xml文件'
psr = et.XMLParser(remove_blank_text = True)
tree = et.parse(capXmlPath, psr)
rt = tree.getroot()
# print et.tostring(tree, pretty_print = True)
node = rt[0][0]
print node.attrib
'总共data条目数量'
dataCnt = len(node)

'记录oni起始绝对时间戳的txt'
oniBegtPath = r'D:\Users\zhangxaochen\Documents\ir-chessboard.oni.txt'
oniBegtPath = r'D:\Users\zhangxaochen\Documents\axxb-data\ir-cb-imu-precalib-20160103\ir-cb-imu-precalib-20160103.oni.txt'
oniBegt = np.asscalar(np.loadtxt(oniBegtPath))
print 'oniBegt: %d'%oniBegt

# openni2.initialize()     # can also accept the path of the OpenNI redistribution

# oniFname = r'D:\Users\zhangxaochen\Documents\ir-chessboard.oni'
# dev = openni2.Device.open_file(oniFname)
# # dev = openni2.Device.open_any()
# print 'dev.get_device_info:', dev.get_device_info()
# print 'dev.get_sensor_info:', dev.get_sensor_info(1)

# depth_stream = dev.create_depth_stream()
# depth_stream.start()
# ir_stream = dev.create_ir_stream()
# ir_stream.start()

rmat0imu = q0imu = None
dQuasImu = []
'初始IMU三轴与cv坐标轴的偏差旋转：'
dQuasImuInit = np.eye(3)

'生成以前协议的 syntheticRT.txt 文件, 共12列: tvec*3, rmat*9; 对应pcl参数 -csv_rt_hint'
imuRt12 = [] #之后转为np-2d-array
'imuCurrPos 记录当前在imu-xml找到第几帧了'
imuCurrPos = 0
depFcnt = depth_stream.get_number_of_frames()
# for i in range(depFcnt):
#     depframe = depth_stream.read_frame()
#     depfid = depframe.frameIndex
#     depRelTs = depframe.timestamp #微秒，非毫秒

#     irframe = ir_stream.read_frame()
#     irfid = irframe.frameIndex
#     irRelTs = irframe.timestamp

'用之前预读得到的：'
for i, irf in enumerate(irfs):
    dataframe = irf #这次用 ir-frame 做主数据流
    fid = dataframe.frameIndex
    relTs = dataframe.timestamp

    print '--:', i, fid

    '获取真实数据帧'
    frame_data = dataframe.get_buffer_as_uint16()
    #dat=np.frombuffer(frame_data, dtype=np.uint16)
    dat=np.array(frame_data) #跟上面等价？
    #print fid, relTs, dat[:20]
    
    absTs = oniBegt + relTs/1e3 #毫秒
    absDt = datetime.fromtimestamp(absTs / 1e3)
    #print absDt
    
    'tvec*3, rmat*9 (共12列); tvec 目前fake: 0,0,0'
    imuRtRow  = [0]*3
    
    '此处故意用下标i, 虽然不好:'
    for j in range(imuCurrPos, dataCnt - 1):
        imuData = node[j]
        attribs = imuData.attrib
        imuDtStr = attribs['realtime']
        imuDtObj = datetime.strptime(imuDtStr, '%Y-%m-%d %H:%M:%S.%f')

        'imu-dt转为ts:'
        #imuAbsTs = time.mktime(imuDtObj.timetuple()) #丢失毫秒
        imuAbsTs = (imuDtObj - datetime(1970,1,1)).total_seconds() * 1e3

        '第 j+1 条目的时间戳'
        imuDtObj2 = datetime.strptime(node[j+1].attrib['realtime'], '%Y-%m-%d %H:%M:%S.%f')

        if imuDtObj <= absDt <= imuDtObj2:
            print '==:', j
            rxyzw = [float(attribs['Rx']), float(attribs['Ry']), float(attribs['Rz']), float(attribs['Rw'])]
            '此矩阵未标定, 坐标系可能与cv坐标系定义不同'
            rmat = tf.q2m(rxyzw)[:3, :3] #本来得到的是 4x4 矩阵
            
            '起始帧:'
            if rmat0imu is None:
                q0imu = rxyzw
                rmat0imu = rmat
                print 'q0imu:', q0imu

            'i转到0: B0.T*Bi'
            dQiImu = tf.qmul(tf.qinv(q0imu), rxyzw)
            dQuasImu.append(dQiImu)
            print 'dQiImu:', dQiImu
            
            imuRtRow.extend(rmat.flatten())
            
            imuCurrPos = j
            imuRt12.append(imuRtRow)
            
            pass
        pass
    
    #imuRt12.append(imuRtRow) #移到内循环 innerFor->if
    pass

dQuasImu = np.asarray(dQuasImu)

# depth_stream.stop()
# ir_stream.stop()
# openni2.unload()

# ---------------
print np.array(imuRt12).shape #==>(N, 12)

'保存到 txt 文件, 需要手动拷贝到oni导出为png序列的目录'
np.savetxt('syntheticRT.txt', imuRt12, fmt='%.12f')

{'frames': '1792', 'phyId': '1191182355'}
oniBegt: 1451794662692
--: 0 1
==: 576
q0imu: [-0.330018341541, -0.163624107838, 0.186878472567, 0.910709321499]
QiImu: [  0.00000000e+00   0.00000000e+00  -2.77555756e-17   1.00000000e+00]
--: 1 2
==: 581
QiImu: [  8.97661531e-04  -1.68305608e-03   8.89482526e-04   9.99997767e-01]
--: 2 3
==: 582
QiImu: [  5.21694596e-04  -1.07718246e-03   1.10791442e-03   9.99998636e-01]
--: 3 4
==: 584
QiImu: [  1.37431879e-04  -2.19137731e-03   3.70413254e-04   9.99997585e-01]
--: 4 5
==: 584
QiImu: [  1.37431879e-04  -2.19137731e-03   3.70413254e-04   9.99997585e-01]
--: 5 6
==: 585
QiImu: [-0.00145469 -0.00203092  0.0016782   0.99999543]
--: 6 7
==: 587
QiImu: [-0.00305804 -0.00130526  0.00267758  0.99999083]
--: 7 8
==: 587
QiImu: [-0.00305804 -0.00130526  0.00267758  0.99999083]
--: 8 9
==: 588
QiImu: [-0.0018305  -0.00219674  0.00146375  0.99999488]
--: 9 10
==: 589
QiImu: [ -6.26571314e-04  -2.33760381e-03   1.30066428e-03   9.99996221e-01]
--: 10 11


In [12]:
plot(dQuasImu)

In [47]:
depth_stream.stop()
openni2.unload()


In [40]:
'''
//2016-1-8 16:07:55
分别使用理想矩阵, 标定矩阵测试IMU姿态转到cv的结果 xbx', 观察方式: 1. 矩阵9元; 2. 四元数
+++++++++++++++
暂不管上面IR-IMU-calib过程，直接用 IMU 数据做初值
读取oni, depth(not IR)为主轴, 提取imu-xml对应时间的数据帧
用到 https://pypi.python.org/pypi/primesense
'''
import numpy as np
from datetime import datetime
from lxml import etree as et
from primesense import openni2

'与oni对应的IMU(zsens采集程序) XML数据'
capXmlPath = r'D:\motion-sensor\zsens\_build.vc11\bin\Release\CaptureSession.ir-chessboard.xml'
'解析CaptureSession.xml文件'
psr = et.XMLParser(remove_blank_text = True)
tree = et.parse(capXmlPath, psr)
rt = tree.getroot()
# print et.tostring(tree, pretty_print = True)
node = rt[0][0]
print node.attrib
'总共data条目数量'
dataCnt = len(node)

'记录oni起始绝对时间戳的txt'
oniBegtPath = r'D:\Users\zhangxaochen\Documents\ir-chessboard.oni.txt'
oniBegt = np.asscalar(np.loadtxt(oniBegtPath))
print 'oniBegt: %d'%oniBegt

openni2.initialize()     # can also accept the path of the OpenNI redistribution

oniFname = r'D:\Users\zhangxaochen\Documents\ir-chessboard.oni'
dev = openni2.Device.open_file(oniFname)
# dev = openni2.Device.open_any()
print 'dev.get_device_info:', dev.get_device_info()
print 'dev.get_sensor_info:', dev.get_sensor_info(1)

depth_stream = dev.create_depth_stream()
depth_stream.start()

'生成以前协议的 syntheticRT.txt 文件, 共12列: tvec*3, rmat*9; 对应pcl参数 -csv_rt_hint'
imuRt12 = [] #之后转为np-2d-array
'imuCurrPos 记录当前在imu-xml找到第几帧了'
imuCurrPos = 0
depFcnt = depth_stream.get_number_of_frames()
for i in range(depFcnt):
    frame = depth_stream.read_frame()
    fid = frame.frameIndex
    depRelTs = frame.timestamp #微秒，非毫秒

    print '--:', i, fid

    '获取真实数据帧'
    frame_data = frame.get_buffer_as_uint16()
    #dat=np.frombuffer(frame_data, dtype=np.uint16)
    dat=np.array(frame_data) #跟上面等价？
    #print fid, depRelTs, dat[:20]
    
    depAbsTs = oniBegt + depRelTs/1e3 #毫秒
    depAbsDt = datetime.fromtimestamp(depAbsTs / 1e3)
    #print depAbsDt
    
    'tvec*3, rmat*9 (共12列); tvec 目前fake: 0,0,0'
    imuRtRow  = [0]*3
    
    '此处故意用下标i, 虽然不好:'
    for i in range(imuCurrPos, dataCnt - 1):
        imuData = node[i]
        attribs = imuData.attrib
        imuDtStr = attribs['realtime']
        imuDtObj = datetime.strptime(imuDtStr, '%Y-%m-%d %H:%M:%S.%f')

        'imu-dt转为ts:'
        #imuAbsTs = time.mktime(imuDtObj.timetuple()) #丢失毫秒
        imuAbsTs = (imuDtObj - datetime(1970,1,1)).total_seconds() * 1e3

        '第 i+1 条目的时间戳'
        imuDtObj2 = datetime.strptime(node[i+1].attrib['realtime'], '%Y-%m-%d %H:%M:%S.%f')

        if imuDtObj <= depAbsDt <= imuDtObj2:
            print '==:', i
            rxyzw = [float(attribs['Rx']), float(attribs['Ry']), float(attribs['Rz']), float(attribs['Rw'])]
            rmat = tf.q2m(rxyzw)[:3, :3] #本来得到的是 4x4 矩阵
            imuRtRow.extend(rmat.flatten())
            
            imuCurrPos = i
            imuRt12.append(imuRtRow)
            
            pass
        pass
    
    #imuRt12.append(imuRtRow) #移到内循环 innerFor->if
    pass

depth_stream.stop()
openni2.unload()

# ---------------
print np.array(imuRt12).shape #==>(N, 12)

'保存到 txt 文件, 需要手动拷贝到oni导出为png序列的目录'
np.savetxt('syntheticRT.txt', imuRt12, fmt='%.12f')

{'frames': '1684', 'phyId': '1191182355'}
oniBegt: 1448203357080
dev.get_device_info: OniDeviceInfo(uri = 'D:\\Users\\zhangxaochen\\Documents\\ir-chessboard.oni', vendor = 'PrimeSense, Ltd.', name = 'oni File', usbVendorId = 0, usbProductId = 0)
dev.get_sensor_info: <primesense.openni2.SensorInfo object at 0x05CBEB70>
--: 0 1
==: 974
--: 1 2
==: 981
--: 2 4
==: 982
--: 3 6
==: 985
--: 4 7
==: 986
--: 5 9
==: 988
--: 6 10
==: 989
--: 7 12
==: 991
--: 8 13
==: 992
--: 9 15
==: 994
--: 10 16
==: 995
--: 11 18
==: 997
--: 12 20
==: 1000
--: 13 21
==: 1001
--: 14 23
==: 1002
--: 15 24
==: 1004
--: 16 26
==: 1005
--: 17 27
==: 1006
--: 18 29
==: 1007
--: 19 30
==: 1009
--: 20 32
==: 1010
--: 21 33
==: 1012
--: 22 35
==: 1015
--: 23 36
==: 1015
--: 24 38
==: 1017
--: 25 39
==: 1017
--: 26 41
==: 1019
--: 27 42
==: 1021
--: 28 44
==: 1022
--: 29 45
==: 1024
--: 30 47
==: 1025
--: 31 48
==: 1027
--: 32 50
==: 1028
--: 33 51
==: 1029
--: 34 53
==: 1031
--: 35 55
==: 1034
--: 36 56
==: 1035
--: 3

In [14]:
imuCurrPos

1613

In [11]:
depFcnt

649

In [9]:
np.array(imuRt12).shape

(649,)

In [5]:
imuCurrPos

0

In [4]:
dataCnt

1684

In [ ]:
cv2.findChessboardCorners

In [22]:
'''
合成数据, 用于测试 axxb.matlab 代码 ①算法正确性; ②调用正确性
存到 axxb.synthetic.txt

1. 之前matlab代码调用时, 用的 A=Ai*A0', B=Bi*B0', 故此处
2. 

'''

# 'rmat: a0, b0'
ra0=tf.random_rotation_matrix() #4x4
# print rot
rb0=tf.random_rotation_matrix() #4x4

# ras=[ra0]
# rbs=[rb0]

e1 = np.array([10,20,30])*np.pi/180
xx = tf.e2m(*e1, axes='rxyz')

axxbCsvMat = []
for i in range(33):
    dr=tf.random_rotation_matrix()
    
    'rmat: ai, bi'
#     rai=ra0*dr #是否dr应左乘？对错不确定
#     rbi=rb0*dr
    
    '''
    必然不是上面统一左/右乘 dr, 
    1. 目前暂时放弃合成数据, 2015-12-7 04:21:18
    2. 用策略： b=dr; a=xbx'; 2015-12-7 09:03:15
    3. 上面错了, 无法验证axxb的a, b到底是姿态量还是delta量；
        改为b=dr*b0; a=xbx'*a0. 已代码确认a,b确实是delta量. 2015-12-10 09:06:59
        
    '''
    
    rbi=dr.dot(rb0)
    #a=xbx'
    rai=xx.dot(rbi).dot(np.linalg.inv(xx)).dot(ra0)
    '''
    //2015-12-19 16:06:28
    对应 dRa=Rai*Ra0', dRb=Rbi*Rb0' 应该是 dRa=X*dRb*X', 之前似乎写错了(写成dRa=X*Rbi*X'), 为什么带入 axxb.m 中验证看起来没错?
    答:
    发现: Rb0=dR0*{Rb0}, Ra0=X*Rb0*X'*{Ra0}, 顺利导致 Rai*Ra0'=【X*Rbi*Rb0'*X'】=X*dRi*dR0'*X', 所以没错
    同时, 若令: Rai=X*dRi*X'*{Ra0}, 也是对的（并不更好）, 因为恰好 Rbi*Rb0'=dRi*dR0'
    最终, 基础方案为: Rai=X*Rbi*X' 就够了！不必管 Rbi 构造方法
    '''
##     rai=xx.dot(dr).dot(np.linalg.inv(xx)).dot(ra0) #也对，暂不用
    
    '''
    //2015-12-19 21:42:23
    改为按照真实情况: IMU的 Rbi 为body-world, 故应为: Rai*Ra0'*X=X*Rbi'*Rb0, ==> Rai=X*Rbi'*X' 即可
    '''
    rai=xx.dot(rbi.T).dot(xx.T)
    
#     ras.append(rai) #目前没用, 2015-12-7 04:53:31
#     rbs.append(rbi)
    
    'rmat-rodrigues'
    rrai,_=cv2.Rodrigues(rai[:3, :3])
    rrbi,_=cv2.Rodrigues(rbi[:3, :3])
    
    'csv-a-row, [ar,at000-fake,br,bt000]'
    row=hstack([rrai.ravel(), np.zeros(3), rrbi.ravel(), np.zeros(3)])
    axxbCsvMat.append(row) #【注意】ra0,rb0 未存入, 因为循环内不好写, 循环外冗长
    pass

axxbCsvMat=np.array(axxbCsvMat)


'存合成文件'
fn=r'axxb.synthetic.txt'
np.savetxt(fn, axxbCsvMat, fmt='%f')#, header=csvHeader)


In [107]:
hstack([rrai.ravel(), rrbi.ravel()])

array([ 0.19678138,  0.02782646,  3.10268862, -0.52526004, -0.52741466,
        0.52923959])

In [101]:
t,_=cv2.Rodrigues(ra0[:3,:3])

In [92]:
cv2.Rodrigues(ra0)

(array([[ 0.],
        [ 0.],
        [ 0.]]),
 array([[  1.98579266e-313,   7.10697189e-294,   4.57862910e-264],
        [  6.87951035e-308,   1.80060130e-316,   4.24531063e-314],
        [  6.36598737e-314,   7.12766127e-294,   2.13741533e+133],
        [  1.27319748e-313,   6.86637331e-313,   2.19874894e-314],
        [  1.80229430e-314,   8.48798316e-314,  -6.25903818e+122],
        [  4.26726213e+133,   1.27319747e-313,   6.88168480e-308],
        [  6.36730611e-314,   4.24531045e-314,   5.00347651e-314],
        [  7.10682784e-294,   3.95495122e+133,   1.40947134e-264],
        [  7.10680395e-294,   1.27319747e-313,   1.69759663e-313]]))

In [13]:
sinfo=dev.get_sensor_info(3)
sinfo.sensorType

OniSensorType.ONI_SENSOR_DEPTH

In [ ]:
dat=np.array(frame_data)
# dat[:44]

In [38]:
dat=np.frombuffer(frame_data, dtype=np.uint16)
dat

array([ 9300, 35612,  9332, ...,     0, 55296, 56776], dtype=uint16)